In [9]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def mod_padding_symmetric(image, factor=64):
    """Padding the image to be divided by factor."""
    height, width = image.shape[0], image.shape[1]
    height_pad, width_pad = ((height + factor) // factor) * factor, (
        (width + factor) // factor
    ) * factor
    padh = height_pad - height if height % factor != 0 else 0
    padw = width_pad - width if width % factor != 0 else 0
    image = tf.pad(
        image, [(padh // 2, padh // 2), (padw // 2, padw // 2), (0, 0)], mode="REFLECT"
    )
    return image

def make_shape_even(image):
    """Pad the image to have even shapes."""
    height, width = image.shape[0], image.shape[1]
    padh = 1 if height % 2 != 0 else 0
    padw = 1 if width % 2 != 0 else 0
    image = tf.pad(image, [(0, padh), (0, padw), (0, 0)], mode="REFLECT")
    return image

def process_image(image):
    input_img = np.asarray(image) / 255.0
    height, width = input_img.shape[0], input_img.shape[1]

    # Padding images to have even shapes
    input_img = make_shape_even(input_img)
    height_even, width_even = input_img.shape[0], input_img.shape[1]

    # Padding images to be multiples of 64
    input_img = mod_padding_symmetric(input_img, factor=64)
    input_img = tf.expand_dims(input_img, axis=0)
    return height, width, input_img

def detect_pixelated_regions(image, block_size=2, threshold=0.8):
    """
    Detects pixelated regions in an image.
    :param image: Input image (assumed to be in BGR format)
    :param block_size: Size of the blocks to check for pixelation
    :param threshold: Threshold to decide if a block is pixelated
    :return: Mask of pixelated regions
    """
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)

    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            block = image[y:y + block_size, x:x + block_size]
            block_mean = np.mean(block, axis=(0, 1))
            block_variance = np.mean((block - block_mean) ** 2)
            if block_variance < threshold:
                mask[y:y + block_size, x:x + block_size] = 255

    return mask

def upsample_pixelated_regions(image, mask, upsample_factor=4):
    """
    Upsamples pixelated regions in the image using bicubic interpolation.
    :param image: Input image (assumed to be in BGR format)
    :param mask: Mask of pixelated regions
    :param upsample_factor: Factor by which to upsample the pixelated regions
    :return: Image with upsampled pixelated regions
    """
    result = image.copy()
    height, width, _ = image.shape
    for y in range(0, height):
        for x in range(0, width):
            if mask[y, x] == 255:
                block_y_start = max(0, y - upsample_factor // 2)
                block_y_end = min(height, y + upsample_factor // 2)
                block_x_start = max(0, x - upsample_factor // 2)
                block_x_end = min(width, x + upsample_factor // 2)
                
                block = image[block_y_start:block_y_end, block_x_start:block_x_end]
                upsampled_block = cv2.resize(block, (block.shape[1] * upsample_factor, block.shape[0] * upsample_factor), interpolation=cv2.INTER_CUBIC)
                
                result[block_y_start:block_y_end, block_x_start:block_x_end] = cv2.resize(upsampled_block, (block.shape[1], block.shape[0]), interpolation=cv2.INTER_CUBIC)
                
    return result


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def evaluate_image(image, upsample_factor, block_size, threshold, ground_truth_mask=None):
    # Process the image (padding)
    height, width, processed_image = process_image(image)

    # Convert processed image back to numpy array for OpenCV processing
    processed_image = processed_image.numpy()[0] * 255

    # Detect pixelated regions
    mask = detect_pixelated_regions(processed_image, block_size=block_size, threshold=threshold)

    # Upsample pixelated regions
    upsampled_image = upsample_pixelated_regions(processed_image.astype(np.uint8), mask, upsample_factor=upsample_factor)

    # Display original pixelated image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.title('Original Pixelated Image')
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Display upsampled image
    plt.subplot(1, 3, 2)
    plt.title(f'Upsampled Image (Factor {upsample_factor})')
    plt.imshow(cv2.cvtColor(upsampled_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Calculate and display PSNR
    psnr_value = psnr(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.cvtColor(upsampled_image, cv2.COLOR_BGR2RGB))
    plt.subplot(1, 3, 3)
    plt.title(f'PSNR: {psnr_value:.2f} dB')
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Display original image again for comparison
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Calculate and display SSIM (optional)
    ssim_value = ssim(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), cv2.cvtColor(upsampled_image, cv2.COLOR_BGR2RGB), multichannel=True)
    print(f'SSIM: {ssim_value:.4f}')

# Load the image
image_path = "C:\\Users\\ROMIT\\OneDrive\\Documents\\Image Denoising\\data\\pixel-barack.png"
image = cv2.imread(image_path)

if image is None:
    raise ValueError("Error loading image. Please check the file path and format.")

# Experiment with different parameters
upsample_factor = 16  # Experiment with values like 2, 4, 8, 16
block_size = 2      # Experiment with values like 2, 4, 8, 16
threshold = 0.6     # Experiment with values like 0.2, 0.4, 0.6, 0.8

evaluate_image(image, upsample_factor, block_size, threshold)
